# 한빛아카데미

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import urllib.request
import pandas as pd
import time

In [2]:
import requests
url = 'https://www.hanbit.co.kr/academy/books/new_book_list.html'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36'}
req = requests.get(url, headers = header)
soup = BeautifulSoup(req.text, 'html.parser')

In [3]:
import re

data = {
    'head' : [],
    'body' : []
}

for i, row in enumerate(soup.select('#container .sub_book_list')):
    
    data['body'].append({})
    
    for col in row.select('.pop_quick_bg'):
        match = re.compile(r"p_code=(\S+\d+)").search(str(col))
        txt = match.group(1) if match != None else 0
        data['body'][i].update({'p_code':txt})

    for col in row.select('.book_tit'):
        txt = col.get_text(strip=True)
        data['head'].append(txt)
        data['body'][i].update({'title':txt})

    for col in row.select('.book_writer'):
        txt = col.get_text(strip=True)
        data['body'][i].update({'writer':txt})

    for col in row.select('.price'):
        txt = col.get_text(strip=True)
        txt = int(txt.replace(',', '').replace('원', ''))
        data['body'][i].update({'price':txt})

In [4]:
[i for i in data['body'] if i.get('p_code') == 'B9686002641']

[]

In [5]:
df = pd.DataFrame(data['body'])
df.sort_values(by="price", ascending=False)
df

,p_code,title,writer,price
0,B4797877052,포르잔 C++ 에센셜,"Behrouz A. Forouzan , Richard F. Gilberg",32000
1,B6503201303,"IT CookBook, 액세스 2019로 배우는 데이터베이스 기초와 실습",김지연,28000
2,B1851418066,포르잔 C++ 바이블,"Behrouz A. Forouzan , Richard F. Gilberg",45000
3,B9479195027,딥러닝을 위한 선형대수학,Gilbert Strang,38000
4,B2082711299,"IT CookBook, 오라클로 배우는 데이터베이스 개론과 실습(2판)","박우창 , 남송휘 , 이현룡",29000
5,B1825116777,대학을 버려야 대학이 산다,조동성 외 인천대학교 가족 178명,18000
6,B5937482728,"IT CookBook, 따라 하면서 배우는 사물인터넷",양재삼,28000
7,B8404138031,R로 하는 다변량 데이터 분석 : 이론부터 실무 활용까지,"카와하시 잇코 , 이와마 노리카즈 , 스즈키 마사유키",32000
8,B8573612142,창의적 공학설계(3판),김은경,23000
9,B8658821149,공학 핵심수학(2판) : 기초부터 심화까지 핵심만 쏙쏙,함남우,23000


## 페이지네이션

In [6]:
import requests
base_url = 'https://www.hanbit.co.kr/academy/books/new_book_list.html'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36'}
req = requests.get(base_url, headers = header)
soup = BeautifulSoup(req.text, 'html.parser')

In [7]:
import re

def get_next_page(soup):
    page_nums = []
    for i, row in enumerate(soup.select('div.paginate > a')):
        match = re.compile(r"page=(\d+)").search(str(row))
        txt = int(match.group(1)) if match != None else 0
        page_nums.append(txt)
    return max(page_nums)

def is_max_page(page):

    page = page + 1
    web_url = base_url + '?page=' + str(page)

    with urllib.request.urlopen(web_url) as response:
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')
        
        nolist = soup.select('#container .sub_book_list_area .no-list')
        islist = (False if len(nolist) == 0 else True)

        return islist

def get_max_page(page):

    page = page + 1
    web_url = base_url + '?page=' + str(page)

    with urllib.request.urlopen(web_url) as response:
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')
        last = get_next_page(soup)

        if is_max_page(last):
            return last
        else:
            return get_max_page(last)

In [8]:
data = {
    'head' : [],
    'body' : []
}

page_max = get_max_page(get_next_page(soup))

for page in range(0, page_max):

    page = page + 1
    web_url = base_url + '?page=' + str(page)

    page_size = len(data['body'])

    with urllib.request.urlopen(web_url) as response:
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')

        for i, row in enumerate(soup.select('#container .sub_book_list')):

            idx = page_size + i
            data['body'].append({})

            for col in row.select('.pop_quick_bg'):
                match = re.compile(r"p_code=(\S+\d+)").search(str(col))
                txt = match.group(1) if match != None else 0
                data['body'][idx].update({'p_code':txt})

            for col in row.select('.book_tit'):
                txt = col.get_text(strip=True)
                data['head'].append(txt)
                data['body'][idx].update({'title':txt})

            for col in row.select('.book_writer'):
                txt = col.get_text(strip=True)
                data['body'][idx].update({'writer':txt})

            for col in row.select('.price'):
                txt = col.get_text(strip=True)
                txt = int(txt.replace(',', '').replace('원', ''))
                data['body'][idx].update({'price':txt})

In [9]:
len(data['body'])

301

In [10]:
df = pd.DataFrame(data['body'])
df.sort_values(by="price", ascending=False)
df

,p_code,title,writer,price
0,B4797877052,포르잔 C++ 에센셜,"Behrouz A. Forouzan , Richard F. Gilberg",32000
1,B6503201303,"IT CookBook, 액세스 2019로 배우는 데이터베이스 기초와 실습",김지연,28000
2,B1851418066,포르잔 C++ 바이블,"Behrouz A. Forouzan , Richard F. Gilberg",45000
3,B9479195027,딥러닝을 위한 선형대수학,Gilbert Strang,38000
4,B2082711299,"IT CookBook, 오라클로 배우는 데이터베이스 개론과 실습(2판)","박우창 , 남송휘 , 이현룡",29000
...,...,...,...,...
296,B5468706684,문제해결력을 키우는 이공계 글쓰기 : 발표와 토론까지,한성우,18000
297,B1672014946,"진로개발과 직업세계 : 대학생과 청년층을 위한 진로, 취업지침서","서일수 , 김도균 , 김지혜 , 이정배",20000
298,B4698900723,Essential Calculus,James Stewart,36000
299,B2947348985,"IT CookBook, 디지털 통신 시스템 : Digital Communicatio...","정호영 , 임승각",25000
